In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

<h1>Load the data</h1>
<p>Load the data</p>
<p>Seperate the target and the inputs</p>

In [4]:
data = pd.read_csv('gtd_clean.csv', encoding='latin1', low_memory=False)


# Seperate the target from the data

targetLabelEncoder = preprocessing.LabelEncoder()
targetLabelEncoder.fit(data['gname'].unique())
target = targetLabelEncoder.transform(data['gname'])

# requireduniquetarget = [data['gname'].unique(), list(range(len(data['gname'].unique())))]

<h1>Preprocessing of Data</h1>
<p>Remove any features that are not required<p>
<p>Set up data into the GTD object<p>

In [5]:
from sklearn.feature_extraction import DictVectorizer
features = data[['iyear', 'country_txt', 'region_txt', 'attacktype1_txt', 'targtype1_txt','weaptype1_txt' ,'extended', 
                 'specificity','vicinity', 'crit1','crit2','crit3', 'multiple', 'success', 'suicide' , 'attacktype2_txt',
                 'attacktype3_txt','targsubtype1_txt', 'natlty1_txt', 'targtype2_txt', 'targsubtype2_txt', 'natlty2_txt',
                'weapsubtype1_txt','weaptype2_txt','weapsubtype2_txt', 'weaptype3_txt']]
# features = data[['iyear', 'country_txt', 'region_txt', 'attacktype1_txt', 'targtype1_txt', 'extended', 'specificity', 'vicinity', 
#                 'crit1','crit2','crit3','doubtterr', 'alternative', 'multiple', 'success', 'suicide', 'attacktype2_txt', 'attacktype3_txt', 
#                 'targsubtype1_txt', 'corp1','target1', 'natlty1_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2','natlty2_txt',
#                  'targtype3_txt', 'targsubtype3_txt', 'corp3', 'target3', 'natlty3', 'natlty3_txt', 'guncertain1', 'guncertain2', 'guncertain3', 
#                  'nperps', 'nperpcap','claimed','claimmode_txt', 'claim2', 'claimmode2_txt', 'claim3','claimmode3_txt', 'compclaim', 'weaptype1_txt',
#                 'weapsubtype1_txt','weaptype2_txt','weapsubtype2_txt', 'weaptype3_txt']] 
# , 'suicide'
# target1 - has approx 2600 categorical data
# 'corp1' - has approx 2900 categorical data

df = (pd.DataFrame(features)).to_dict(orient='records')
dv = DictVectorizer(sparse=False) 
features = dv.fit_transform(features.to_dict(orient='records'))

print(features)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [6]:
np.shape(features)

(153011, 965)

<h1>Apply PCA</h1>
<p> Reduce dimensionality of the features
</p>

In [29]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
reducedFeatures = pca.fit_transform(features)

<h1> Split the data into test and train set for PCA features</h1>

In [30]:
from sklearn.model_selection import train_test_split
X_trainp, X_testp, Y_trainp, Y_testp = train_test_split(reducedFeatures, target, test_size=0.30)

<h1>Apply Truncated SVD</h1>
<p> Reducing dimensionality of the features by applying Truncated SVD
</p>

In [31]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=200)
reducedSVDFeatures = svd.fit_transform(features)

<h1> Split the data into test and train set for Truncated SVD features</h1>

In [32]:
from sklearn.model_selection import train_test_split
X_traint, X_testt, Y_traint, Y_testt = train_test_split(reducedSVDFeatures, target, test_size=0.30)

<h1>Random Forest / PCA</h1>
<p> Fit the data and target using Random forest classifier
</p>

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=5, max_depth=4, min_samples_split=2)
scores = cross_val_score(clf, X_trainp, Y_trainp)
scores.mean() 

//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


0.50149875934534816

<h1>Random Forest / Truncated SVD</h1>
<p> Fit the data and target using Random forest classifier
</p>

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=5, max_depth=4, min_samples_split=2)
scores = cross_val_score(clf, X_traint, Y_traint)
scores.mean() 

//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


0.50421464898015644